In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from appaloosa import RunLC
import os
import numpy as np
import pyke as pk
import pandas as pd
oid=211410963
loc = 'stars_shortlist/M67'
tpf_loc = 'stars_shortlist/M67/ktwo{}-c05_lpd-targ.fits.gz'.format(oid)
lc_loc = 'stars_shortlist/M67/hlsp_everest_k2_llc_{}-c05_kepler_v2.0_lc.fits'.format(oid)


In [ ]:
os.chdir('/home/ekaterina/Documents/appaloosa')
RunLC(lc_loc, dbmode='everest', display=True, debug=True, dofake=False, writeout=True)

In [2]:
os.chdir('/home/ekaterina/Documents/appaloosa/stars_shortlist/M67/results/test_11')
flux = pd.read_csv(str(oid)+'_flux.csv')#,usecols=['time','flux_gap','flux_model'])
flares = pd.read_csv(str(oid)+'_flares.csv')#,usecols=['istart','istop'])
#flux.plot(x='time',y=['flux_gap','flux_model'])
#flux.plot(x='time',y='flux_model')
# for index, row in flares.iterrows():
#     print(flux.loc[row.istart:row.istop])
flux.head()

,Unnamed: 0,flux_gap,flux_model,time
0,0,602.901787,606.787976,2306.611021
1,1,606.946295,609.228221,2306.631453
2,2,604.881634,610.677717,2306.651885
3,3,607.185952,611.281474,2306.672317
4,4,602.350398,611.184463,2306.692750


In [6]:
os.chdir('/home/ekaterina/Documents/appaloosa/stars_shortlist/M67')
tpf = pk.KeplerTargetPixelFile('ktwo{}-c05_lpd-targ.fits.gz'.format(oid))
apmask = tpf.pipeline_mask*0.01
#d = tpf.flux*tpf.pipeline_mask
tpf.time


array([ 2306.61102064,  2306.63145297,  2306.65188509, ...,  2381.39138211,
        2381.41181404,  2381.43224587])

In [11]:
import math
import timeit
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]
    
flux['tpf_time'] = 'Nan'
for id_,row in flux.iterrows():
    try:
        flux['tpf_time'].iloc[id_] = find_nearest(tpf.time, row.time)
        #timeit.timeit(find_nearest(tpf.time, row.time))
        #flux['tpf_time'].iloc[id_] = tpf.time[np.argmin(abs(tpf.time[id_:]-row.time))]
        #print(tpf.time[max(0,id_-5):min(id_+5,len(tpf.time))])
    except ValueError:
        pass
    

flux.iloc[645:655]

,Unnamed: 0,flux_gap,flux_model,time,tpf_time
645,645,597.718434,600.178740,2319.789712,2319.77
646,646,597.603123,599.826590,2319.810144,2319.81
647,647,598.377805,599.522437,2319.830576,2319.83
648,648,593.212411,599.292055,2319.851008,2319.85
649,649,601.506722,599.161222,2319.871440,2319.87
650,650,601.171965,599.155651,2319.891872,2319.89
651,651,605.808730,599.284957,2319.912304,2319.91
652,652,598.498308,599.521451,2319.932736,2319.93
653,653,601.549201,599.832178,2319.953168,2319.95
654,654,603.733520,600.184182,2319.973600,2319.97


In [17]:
flux['isflare'] = False
print(flux.head())
for id_, row in flares.iterrows():
    flux['isflare'].iloc[row.istart:row.istop+1] = True
flux.to_csv('{}_tpf_times_FLUX.csv'.format(oid))
#flux.iloc[1574:1603]

   Unnamed: 0    flux_gap  flux_model       time tpf_time  isflare
0           0  602.901787  606.787976  2306.6110  2306.61    False
1           1  606.946295  609.228221  2306.6315  2306.63    False
2           2  604.881634  610.677717  2306.6519  2306.65    False
3           3  607.185952  611.281474  2306.6723  2306.67    False
4           4  602.350398  611.184463  2306.6927  2306.69    False


In [16]:
flux_648 = pd.read_csv('211410963_tpf_times_FLUX.csv')
flux_648.time = flux_648.time.round(decimals=4)
flux.time = flux.time.round(decimals=4)
a = flux.merge(flux_648, on='time')
#print(flux.head())
#print(flux_648.head())
#print(a.head())
s = a[(a.isflare_x == a.isflare_y)]
#print(s.info())
p = a[(a.isflare_x == True)]
#print(p.info())
t = s[s.isflare_x == True]
#print(t)

In [14]:
select_isflare = flux[flux.isflare==True]
isflare_tpf_times = select_isflare.tpf_time.tolist()

for i in isflare_tpf_times[35:65]:
    id_ = np.where(tpf.time == i)[0][0]
    print(id_)

1268
1269
1270
1271
1272
1273
1274
1275
1450
1451
1452
1597
1598
1599
1600
1743
1743
1744
1817
1818
1819
1820
1821
1822
1829
1830
1831
1832
1833
1834


In [25]:
import k2flix
tpf2 = k2flix.TargetPixelFile('ktwo{}-c05_lpd-targ.fits.gz'.format(oid))
for i in range(1508,1530):
    print(tpf2.quality_flags(i))
    

In [25]:
tpf2.save_movie(output_fn='test.gif', start=1508, stop=1530, step=1,
               binning=1, fps=3, dpi=None, stretch='log',
               min_cut=None, max_cut=None, min_percent=60, max_percent=100,
               cmap='gray', time_format='frameno', show_flags=True, data_col='FLUX',
               ignore_bad_frames=True,)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Cosmic ray']
['Thruster firing']
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

#fig, ax = plt.subplots()
#print(type(ax))
#fig.set_tight_layout(True)

# Query the figure's on-screen size and DPI. Note that when saving the figure to
# a file, we need to provide a DPI for that separately.

# Plot a scatter that persists (isn't redrawn) and the initial line.
#x = np.arange(0, 20, 0.1)
#ax.scatter(x, x + np.random.normal(0, 3.0, len(x)))
#line, = ax.plot(x, x - 5, 'r-', linewidth=2)
#ax = tpf.plot(0)[1]
fig, line = tpf.plot(0)
print(type(line))
print('fig size: {0} DPI, size in inches {1}'.format(
    fig.get_dpi(), fig.get_size_inches()))

def update(i):
    label = 'timestep {}'.format(i)
    print(label)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    #line.set_ydata(x - 5 + i)
    #print(type(fig),type(line))
    figure, line = tpf.plot(i)
    print(figure,fig, line)
    line.set_xlabel(label)
    return line, figure

if __name__ == '__main__':
    # FuncAnimation will call the 'update' function for each frame; here
    # animating over 10 frames, with an interval of 200ms between frames.
    anim = FuncAnimation(fig, update, frames=np.arange(0, 10))#, interval=200)
#     if len(sys.argv) > 1 and sys.argv[1] == 'save':
    anim.save('line.gif', dpi=80, writer='imagemagick')
#     else:
#         # plt.show() will just loop the animation forever.
    plt.show()

In [ ]:
print(type(fig),type(line))

In [ ]:
from IPython.display import Image
with open('line.gif','rb') as f:
    display(Image(data=f.read(), format='png'))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#plot = tpf.plot(1200, aperture_mask=apmask)
#plot[0]

for i in isflare_tpf_times[:3]:
    id_ = np.where(tpf.time == i)[0][0]
    print(id_)
    #print(type(tpf.plot(id_, aperture_mask=apmask)[1]))
